In [1]:
from minio import Minio
import random
import struct
import pandas as pd
import boto3
import hashlib
import gmpy2

In [2]:
#Реализация работы DHT протокола

K = 20 # Кол-во ближайших узлов, которые необходимо хранить в таблице маршрутпизатора
ALPHA = 3  # Количество узлов, к которым нужно отправлять запросы на каждом шаге

# Класс узла Kademlia
class Node:
    def __init__(self, ip, port):
        self.id = bytes.fromhex(hashlib.sha256(str(random.uniform(0, 160)).encode()).hexdigest())[:160 // 8].hex()
        self.ip = ip
        self.port = port

    def get_document_id(self, document_name):
        """Вычисляет ID документа."""
        return bytes.fromhex(hashlib.sha256(document_name.encode()).hexdigest())[:160//8].hex()

    def distance(self, other_id):
        """Вычисляет расстояние между двумя узлами (XOR-расстояние идентификаторов)."""
        return int(self.id, 16) ^ int(other_id, 16)

# Класс таблицы маршрутизации Kademlia
class RoutingTable:
    def __init__(self, node):
        self.node = node
        self.buckets = [[] for _ in range(160)]  # 160 бакетов для 160-битных ID

    def add_node(self, node):
        """Добавляет узел в таблицу маршрутизации."""
        bucket_index = self._get_bucket_index(node.id)
        bucket = self.buckets[bucket_index]
        if node not in bucket:
            bucket.append(node)
            #bucket.sort()  # Сортировка по ID для удобства
            self._trim_bucket(bucket)
    def remove_node(self, node):
        bucket_index = self._get_bucket_index(node.id)
        bucket = self.buckets[bucket_index]
        if node in bucket:
            bucket.pop(bucket.index(node))

    def find_closest_nodes(self, target_id, count=K):
        """Возвращает список самых ближайших узлов к указанному ID."""
        closest_nodes = []
        for bucket in self.buckets:
            for node in bucket:
                distance = node.distance(target_id)
                closest_nodes.append((distance, node))
        closest_nodes.sort()
        return [node for _, node in closest_nodes[:count]]

    def _get_bucket_index(self, node_id):
        """Возвращает индекс бакета для указанного ID."""
        return (int(self.node.id, 16) ^ int(node_id, 16)).bit_length() - 1

    def _trim_bucket(self, bucket):
        """Обрезает бакет до максимального количества узлов K."""
        while len(bucket) > K:
            bucket.pop()

# Функция для поиска узла, имеющего файл по алгоритму Kademli

In [2]:
#Реализация Minio S3 клиента
class Minio_client:
    def __init__(self, ip, access_key, secret_key, port):
        self.ip = ip
        self.port = port
        self.access_key = access_key
        self.secret_key = secret_key
        self.self_client = boto3.client(
    's3',
    aws_access_key_id=self.access_key,
    aws_secret_access_key=self.secret_key,
    endpoint_url='http://{}:{}'.format(self.ip, self.port),
    )
        self.other_client = boto3.client(
    's3',
    aws_access_key_id=self.access_key,
    aws_secret_access_key=self.secret_key,
    endpoint_url='http://{}:{}'.format(self.ip, self.port)
    )
    
    def create_self_bucket(self, bucket_name):
        try:
            self.self_client.create_bucket(Bucket=bucket_name)
            return 'Success!'
        except ClientError:
            return None
            
    def remove_self_bucket(self, bucket_name):
        objects = self.self_client.list_objects_v2(Bucket=bucket_name)
        if (objects['KeyCount'] != 0):
            for obj in objects['Contents']:
                self.self_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
        try:
            self.self_client.delete_bucket(Bucket=bucket_name)
        except ClientError:
            return None
            
    def get_from_self_bucket(self, bucket_name, file_name):
        try:
            context = self.self_client.download_file(bucket_name, file_name, file_name)
        except ClientError:
            context = None
        return context
    def remove_obj_from_bucket(self, bucket_name, file_name, self_bucket=False):
        if (self_bucket):
            try:
                self.other_client.delete_object(Bucket=bucket_name, Key=file_name)
                return "Success!"
            except ClientError:
                return None
        else:
            try:
                self.self_client.delete_object(Bucket=bucket_name, Key=file_name)
                return "Success!"
            except ClientError:
                return None
                
    def put_from_self_bucket(self, bucket_name, file_name):
        with open(file_name, "rb") as file:
            self.self_client.upload_fileobj(file, bucket_name, file_name)
        file.close()
    
    def change_other_client(other_ip, other_access_key, other_secret_key):
        self.other_client = boto3.client(
            's3',
            aws_access_key_id=other_access_key,
            aws_secret_access_key=other_secret_key,
            endpoint_url='http://{}:9000'.format(self.ip)
        )
        
    def get_from_other_bucket(self, bucket_name, file_name):
        try:
            context = self.other_client.download_file(bucket_name, file_name, file_name)
        except ClientError:
            context = None
        return context
        
    def put_from_other_bucket(self, bucket_name, file_name, metadata):
        with open(file_name, "rb") as file:
            self.other_client.upload_fileobj(file, bucket_name, file_name)
        file.close()    

    # Функция для поиска узла, имеющего файл по алгоритму Kademli
    def find_node(current_node, target_id, bucket_name, file_name, lhfile_name):
        global routing_table
        """
        Ищет узел с указанным ID в таблице маршрутизации
        используя алгоритм Kademlia.
    
        Args:
            current_node: Текущий узел.
            routing_table: Таблица маршрутизации текущего узла.
            target_id: ID узла, которого нужно найти.
    
        Returns:
            Узел с указанным ID, если найден, иначе None.
        """
    
        closest_nodes = routing_table.find_closest_nodes(target_id)
        tries = 0
        while (tries < 5):
            # Выбираем ALPHA узлов из списка самых близких
            if len(closest_nodes) != 0:
                selected_nodes = closest_nodes[0]
    
            # Отправляем запросы на поиск узла
            change_other_client(closest_nodes.ip, self.access_key, self.secret_key)
            if target_id in self.other_client.list_object_v2(Bucket=bucket_name):
                self.get_from_other_bucket(bucket_name, file_name)
                break
            else:
                self.get_from_other_bucket(bucket_name, lhfile_name, lhfile_name)
                localhosts = pd.read_csv(lhfile_name)
                for index in range(len(localhosts)):
                    routing_table.add_Node(Node(localhosts['id'][index], localhosts['ip'][index], localhosts['port'][index]))
            # Обновляем список самых близких узлов
            closest_nodes = routing_table.find_closest_nodes(target_id)
    
        # Если не найден узел, возвращаем None
        return None

In [ ]:
# node = Node('192.168.13.59', 8000)
# doc_id = node.get_document_id('test') # Работает '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b'
# distance = node.distance(doc_id)

In [ ]:
# node1 = Node('192.168.13.52', 8000)
# node2 = Node('192.168.13.54', 8000)
# node3 = Node('192.168.13.53', 8000)
# node4 = Node('192.168.13.55', 8000)

In [ ]:
# Routing_t = RoutingTable(node)

In [ ]:
# Routing_t.add_node(node4)

In [ ]:
# Routing_t.find_closest_nodes(doc_id)

In [ ]:
# Routing_t.remove_node(node4)

In [6]:
# client = Minio_client('192.168.13.59', 'AFbT5JtHkDPjsaSwSwzo', 'AvW3kMeQLZgdfbuyosPEQ3EIRCc9Ru5JUQ33tYqX', 8000)

In [7]:
# client.create_self_bucket("new-bucket")

'Success!'

In [ ]:
# 'http://{}:{}'.format(client.ip, client.port)

In [19]:
# res = client.get_from_self_bucket('mybucket', 'example.txt')

In [28]:
# client.put_from_self_bucket('new-bucket', 'example.txt')

In [ ]:
# client.remove_self_bucket('new-bucket')

In [27]:
# client.remove_obj_from_bucket('new-bucket', 'example.txt', True)

'Success!'

In [30]:
# client.get_from_self_bucket('new-bucket', 'example.txt')

In [31]:
# client.self_client.download_file('new-bucket', 'example.txt', 'example.txt')